In [3]:
from data_loader_df import *
import sys
import numpy
import pandas as pd

In [5]:
train_file_path = "../data/adult.data"
test_file_path = "../data/adult.test"
train_X, train_Y, _, __, test_X, test_Y = load_all_data(train_file_path, test_file_path, 0, False)
print (train_X)

/home/mke2/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


[[ 1.77428872 -1.1498391  -0.43812161 ...  1.          0.
   0.        ]
 [ 0.86641684  0.24853991 -0.43812161 ...  1.          0.
   0.        ]
 [-0.19276702 -0.04098651 -0.43812161 ...  1.          0.
   0.        ]
 ...
 [-0.64670296  0.39311853 -0.43812161 ...  1.          0.
   0.        ]
 [-0.64670296  0.07405729 -0.43812161 ...  1.          0.
   0.        ]
 [ 0.94207283 -1.44358749 -0.43812161 ...  1.          0.
   0.        ]]
